In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## In this Jupyter Notebook we will display the results after training of the model

### Setting up the environment

In [2]:
!pip install -r requirements.txt

ERROR: Could not find a version that satisfies the requirement python==3.10 (from versions: none)
ERROR: No matching distribution found for python==3.10


In [35]:
!pip uninstall -y numpy
!pip install numpy==1.24.4
!pip install dicom2nifti


Found existing installation: numpy 1.26.4
Uninstalling numpy-1.26.4:
  Successfully uninstalled numpy-1.26.4
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 109.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
thinc 8.3.5 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.24.4 which is incompatible.
jaxlib 0.5.1 requires numpy>=1.25, but you have numpy 1.24.4 which is incompatible.
tensorflow 2.18.0 requires numpy<2.1.0,>=1.26.0, but you have numpy 1.24.4 which is incompatible.
blosc2 3.3.4 requires numpy>=1.26, but you have numpy 1.24.4 which is incompatible.
jax 0.5.2 requires numpy>=1.25, but you have numpy 1.24.4 which is incompatible.
xarray-einstats 0.9.0 requires numpy>=1.25, but you have numpy 1.24.4 which is incompatible.
treescope 0.1.9 requires numpy>=1.25.2, but you have numpy 1.24.4 which is incompatible.
pymc 5.23.0 requ

### Importing the libraries

In [34]:
import os
from glob import glob
import shutil
import dicom2nifti
import nibabel as nib
import numpy as np

from scripts.prepare import divide_groups, convert_nif, remove_empty_groups

ModuleNotFoundError: No module named 'numpy.char'

##### Setting the path to the working directories

In [ ]:
# The input paths for the dicom files
in_path = ['data_set/dicom_files_testing/images',
            'data_set/dicom_files_testing/labels',
            'data_set/dicom_files_training/images',
            'data_set/dicom_files_training/labels',
            'data_set/dicom_files_validation/images',
            'data_set/dicom_files_validation/labels']

# The output paths for the dicom files split into equal groups
group_path = ['data_set_group/dicom_files_testing/images',
            'data_set_group/dicom_files_testing/labels',
            'data_set_group/dicom_files_training/images',
            'data_set_group/dicom_files_training/labels',
            'data_set_group/dicom_files_validation/images',
            'data_set_group/dicom_files_validation/labels']

# The output paths for the nifti files
nif_path = ['data_set_group_nif/nif_files_testing/images',
            'data_set_group_nif/nif_files_testing/labels',
            'data_set_group_nif/nif_files_training/images',
            'data_set_group_nif/nif_files_training/labels',
            'data_set_group_nif/nif_files_validation/images',
            'data_set_group_nif/nif_files_validation/labels']

##### Divide pacients into groups (prepare data)

In [ ]:
divide_groups(in_path, group_path, 16)

convert_nif(group_path, nif_path)

remove_empty_groups(nif_path)

##### Divide pacients into groups (prepare data)

##### Loading the model

In [ ]:
# Load the model
model = UNet(
    spatial_dims=3,
    in_channels=1,
    out_channels=2,
    channels=(16, 32, 64, 128, 256),
    strides=(2, 2, 2, 2),
    num_res_units=2,
    norm=Norm.BATCH,
).to(device)

model.load_state_dict(torch.load(os.path.join(model_path, 'best_metric_model.pth')))

# Set the model to evaluation mode
model.eval()